In [1]:
import numpy as np
import tensorflow as tf
import os

2025-06-04 10:06:50.284485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 10:06:50.524116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-04 10:06:50.524175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-04 10:06:50.573235: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 10:06:50.650120: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# import the train and validation data
all_train_data = os.listdir("seq_2_seq_train_data")

In [3]:
# get the inputs and targets separated
all_train_input_files = [file for file in all_train_data if file.split('.')[0][-14:] == 'input_features']
all_train_target_files = [file for file in all_train_data if file.split('.')[0][-15:] == 'target_features']

In [4]:
all_train_input_files.sort()
all_train_input_files

['block_0101_extracted_input_features.npy',
 'block_0102_extracted_input_features.npy',
 'block_0203_extracted_input_features.npy']

In [5]:
all_train_target_files.sort()
all_train_target_files

['block_0101_extracted_target_features.npy',
 'block_0102_extracted_target_features.npy',
 'block_0203_extracted_target_features.npy']

In [6]:
# load the files
loaded_all_train_input_files = [np.load(os.path.join("seq_2_seq_train_data", file)) for file in all_train_input_files]
loaded_all_train_target_files = [np.load(os.path.join("seq_2_seq_train_data", file)) for file in all_train_target_files]

In [7]:
# stack the files in the list
X_train = np.vstack(loaded_all_train_input_files)

In [8]:
X_train.shape

(2730, 13, 32)

In [9]:
y_train = np.vstack(loaded_all_train_target_files)

In [10]:
y_train.shape

(2730, 7, 32)

In [11]:
# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer for the encoder
    inputs = tf.keras.layers.Input(shape=(input_timesteps, input_features))

    # Encoder LSTM
    encoder = tf.keras.layers.LSTM(64, activation='relu', return_state=True, return_sequences=False)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # Decoder LSTM: We now provide the encoder's state and initialize it with the encoder's final states
    # We reshape the output of the encoder to make sure it is in the expected form for the decoder
    decoder_input = tf.keras.layers.RepeatVector(output_timesteps)(encoder_outputs)

    # Decoder LSTM, where the output sequence length is `output_timesteps` (7)
    decoder_lstm = tf.keras.layers.LSTM(64, activation='relu', return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_input, initial_state=[state_h, state_c])

    # Dense layer to predict the next 7 time periods (each with 32 features)
    outputs = tf.keras.layers.Dense(input_features)(decoder_outputs)

    # Create the model
    model = tf.keras.models.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 13  # 12 time periods
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


2025-06-04 10:07:02.429537: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 13, 32)]             0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['input_1[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 repeat_vector (RepeatVecto  (None, 7, 64)                0         ['lstm[0][0]']                
 r)                                                                                           

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30


2025-06-04 10:07:06.125163: I external/local_xla/xla/service/service.cc:168] XLA service 0x14a160c8dbd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-04 10:07:06.125199: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-06-04 10:07:06.160703: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-04 10:07:06.249025: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1749049626.482196  168195 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


86/86 [==============================] - 6s 29ms/step - loss: 0.1656
Epoch 2/30
86/86 [==============================] - 3s 29ms/step - loss: 0.1035
Epoch 3/30
86/86 [==============================] - 2s 29ms/step - loss: 0.1022
Epoch 4/30
86/86 [==============================] - 2s 28ms/step - loss: 0.1015
Epoch 5/30
86/86 [==============================] - 3s 29ms/step - loss: 0.1008
Epoch 6/30
86/86 [==============================] - 3s 29ms/step - loss: 0.1010
Epoch 7/30
86/86 [==============================] - 3s 29ms/step - loss: 0.1003
Epoch 8/30
86/86 [==============================] - 3s 29ms/step - loss: 0.1007
Epoch 9/30
86/86 [==============================] - 3s 30ms/step - loss: 0.0995
Epoch 10/30
86/86 [==============================] - 3s 30ms/step - loss: 0.0994
Epoch 11/30
86/86 [==============================] - 3s 30ms/step - loss: 0.0989
Epoch 12/30
86/86 [==============================] - 2s 29ms/step - loss: 0.0991
Epoch 13/30
86/86 [=============================